# Spike to Spike Regression: Oxford

In [1]:
import os
import h5py
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader

# Import Slayer from lava-dl
import lava.lib.dl.slayer as slayer

import IPython.display as display
from matplotlib import animation

No CUDA runtime is found, using CUDA_HOME='/usr'


# Create Dataset
Create a simple PyTorch dataset class.
It shows usage of the `slayer.io` module, providing a way to:
- Easily represent events including graded spikes
- Read/write events in different known binary and numpy formats
- Transform event to tensor for processing it using slayer network and convert a spike tensor back to event
- Display/animate the tensor for visualization

In [2]:
class OxfordDataset(Dataset):
    def __init__(self):
        super(OxfordDataset, self).__init__()
        self.input = slayer.io.read_1d_spikes('input.bs1')
        self.target = slayer.io.read_1d_spikes('output.bs1')
        self.target.t = self.target.t.astype(int)

    def __getitem__(self, _):
        return (
            self.input.fill_tensor(torch.zeros(1, 1, 200, 2000)).squeeze(),     # input dimensions ( batch, channel, number of neurons (200), time (2000ms) )
            self.target.fill_tensor(torch.zeros(1, 1, 200, 2000)).squeeze(),    # target dimensions ( batch, channel, number of neurons (200), time(2000ms) )
        )
    
    def __len__(self):
        return 1    # just one sample for this problem

# Create Network
A slayer network definition follows the standard PyTorch way using `torch.nn.Module`.

The network can be described with a combination of individual `synapse`, `dendrite`, `neuron` and `axon` components. 

For rapid and easy development, slayer provides a block interface - `slayer.block` - which bundles all these individual components into a single unit.

In [3]:
class Network(torch.nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        
        neuron_params = {
            'threshold': 0.1,
            'current_decay': 1,
            'voltage_decay': 0.1,
            'requires_grad': True,
        }

        # Define the network
        # CUBA LIF Dense Block documentation (https://lava-nc.org/lava-lib-dl/slayer/block/block.html#lava.lib.dl.slayer.block.cuba.Dense)
        self.blocks = torch.nn.ModuleList([
            slayer.block.cuba.Dense(neuron_params, 200, 256),   # 200 input neurons, 256 output neurons
            slayer.block.cuba.Dense(neuron_params, 256, 200),   # 256 input neurons, 200 output neurons
        ])

    def forward(self, spike):
        for block in self.blocks:
            spike = block(spike)
        return spike
    
    def export_hdf5(self, filename):
        # Export the network parameters to a HDF5 file
        h = h5py.File(filename, 'w')
        layer = h.create_group('layer')

        for idx, block in enumerate(self.blocks):
            block.export_hdf5(layer.create_group(f"{idx}"))

# Instantiate Network, Optimizer, DataSet and DataLoader
Running the network in GPU is as simple as selecting `torch.device('cuda')`.

In [4]:
trained_folder = 'Trained'
os.makedirs(trained_folder, exist_ok=True)

# device = torch.device('cpu')
device = torch.device('cuda')

net = Network().to(device)

lr = 0.001  # learning rate
weight_decay = 1e-5 
optimizer = torch.optim.Adam(net.parameters(), lr=lr, weight_decay=weight_decay)  

training_set = OxfordDataset()    # Create the dataset
train_loader = DataLoader(dataset=training_set, batch_size=1)   # Create the dataloader

RuntimeError: No CUDA GPUs are available